In [22]:
import pandas as pd
import numpy as np
import re

# Dataset con problemas comunes de calidad de datos
empleados_sucio = pd.DataFrame({
    'empleado_id': ['E001', 'E002', 'E003', 'E004', 'E005', 'E006'],
    'nombre': ['Ana García  ', 'carlos lopez', 'MARÍA RODRÍGUEZ', 'Juan Pérez', 'laura  martin', 'Pedro Sánchez'],
    'email': ['ana.garcia@company.com', 'CARLOS.LOPEZ@COMPANY.COM', 'maria.rodriguez@company.com', 
              'juan.perez@company.COM', 'laura.martin@company.com', 'pedro.sanchez@COMPANY.com'],
    'departamento': ['Consultoría', 'It', 'CONSULTORIA', 'Finanzas', 'IT', 'consultoría'],
    'salario': ['65,000', '70000', '62000', '75,000', '68000', '67000'],
    'fecha_ingreso': ['15/01/2020', '2021-03-10', '10-02-2022', '2019/12/05', '2023-01-15', '2020/08/20'],
    'telefono': ['91-555-0123', '91 555 0124', '915550125', '91.555.0126', '91-555-0127', '915550128'],
    'estado': ['activo', 'ACTIVO', 'Activo', 'inactivo', 'INACTIVO', 'Activo']
})

print("Datos originales con problemas de calidad:")
print(empleados_sucio)

Datos originales con problemas de calidad:
  empleado_id           nombre                        email departamento  \
0        E001     Ana García         ana.garcia@company.com  Consultoría   
1        E002     carlos lopez     CARLOS.LOPEZ@COMPANY.COM           It   
2        E003  MARÍA RODRÍGUEZ  maria.rodriguez@company.com  CONSULTORIA   
3        E004       Juan Pérez       juan.perez@company.COM     Finanzas   
4        E005    laura  martin     laura.martin@company.com           IT   
5        E006    Pedro Sánchez    pedro.sanchez@COMPANY.com  consultoría   

  salario fecha_ingreso     telefono    estado  
0  65,000    15/01/2020  91-555-0123    activo  
1   70000    2021-03-10  91 555 0124    ACTIVO  
2   62000    10-02-2022    915550125    Activo  
3  75,000    2019/12/05  91.555.0126  inactivo  
4   68000    2023-01-15  91-555-0127  INACTIVO  
5   67000    2020/08/20    915550128    Activo  


Limpieza y Estandarización de Texto

Normalización de Nombres

In [23]:
# Crear una copia para trabajar
empleados_limpio = empleados_sucio.copy()

# Limpiar espacios en blanco y estandarizar capitalización
empleados_limpio['nombre'] = empleados_limpio['nombre'].str.strip()  # Quitar espacios
empleados_limpio['nombre'] = empleados_limpio['nombre'].str.title()  # Capitalización correcta

print("Nombres después de limpieza:")
print(empleados_limpio[['empleado_id', 'nombre']])

Nombres después de limpieza:
  empleado_id           nombre
0        E001       Ana García
1        E002     Carlos Lopez
2        E003  María Rodríguez
3        E004       Juan Pérez
4        E005    Laura  Martin
5        E006    Pedro Sánchez


Estandarización de Emails

In [24]:
# Convertir emails a minúsculas
empleados_limpio['email'] = empleados_limpio['email'].str.lower()

print("Emails estandarizados:")
print(empleados_limpio[['empleado_id', 'email']])

Emails estandarizados:
  empleado_id                        email
0        E001       ana.garcia@company.com
1        E002     carlos.lopez@company.com
2        E003  maria.rodriguez@company.com
3        E004       juan.perez@company.com
4        E005     laura.martin@company.com
5        E006    pedro.sanchez@company.com


Normalización de Departamentos

In [25]:
# Crear diccionario de mapeo para departamentos
mapeo_departamentos = {
    'consultoría': 'Consultoría',
    'CONSULTORIA': 'Consultoría',
    'it': 'IT',
    'IT': 'IT',
    'finanzas': 'Finanzas'
}

# Aplicar mapeo después de normalizar
empleados_limpio['departamento_normalizado'] = (
    empleados_limpio['departamento']
    .str.lower()
    .str.strip()
    .replace({
        'consultoría': 'Consultoría',
        'consultoria': 'Consultoría',
        'it': 'IT',
        'finanzas': 'Finanzas'
    })
)

print("Departamentos normalizados:")
print(empleados_limpio[['departamento', 'departamento_normalizado']])

Departamentos normalizados:
  departamento departamento_normalizado
0  Consultoría              Consultoría
1           It                       IT
2  CONSULTORIA              Consultoría
3     Finanzas                 Finanzas
4           IT                       IT
5  consultoría              Consultoría


Transformación de Tipos de Datos

In [26]:
# Limpiar formato de salarios y convertir a numérico
def limpiar_salario(salario_str):
    """Convierte string de salario a float, removiendo comas y espacios."""
    if isinstance(salario_str, str):
        # Remover comas y espacios
        salario_limpio = salario_str.replace(',', '').replace(' ', '')
        return float(salario_limpio)
    return float(salario_str)

empleados_limpio['salario_numerico'] = empleados_limpio['salario'].apply(limpiar_salario)

print("Salarios convertidos:")
print(empleados_limpio[['empleado_id', 'salario', 'salario_numerico']])

Salarios convertidos:
  empleado_id salario  salario_numerico
0        E001  65,000           65000.0
1        E002   70000           70000.0
2        E003   62000           62000.0
3        E004  75,000           75000.0
4        E005   68000           68000.0
5        E006   67000           67000.0


Estandarización de Fechas

In [27]:
# Función para normalizar diferentes formatos de fecha
def estandarizar_fecha(fecha_str):
    """Convierte diferentes formatos de fecha a datetime."""
    fecha_str = str(fecha_str).strip()
    
    # Intentar diferentes formatos
    formatos = ['%d/%m/%Y', '%Y-%m-%d', '%d-%m-%Y', '%Y/%m/%d']
    
    for formato in formatos:
        try:
            return pd.to_datetime(fecha_str, format=formato)
        except ValueError:
            continue
    
    # Si ningún formato funciona, usar parser automático
    try:
        return pd.to_datetime(fecha_str)
    except:
        return pd.NaT

empleados_limpio['fecha_ingreso_std'] = empleados_limpio['fecha_ingreso'].apply(estandarizar_fecha)

print("Fechas estandarizadas:")
print(empleados_limpio[['empleado_id', 'fecha_ingreso', 'fecha_ingreso_std']])

Fechas estandarizadas:
  empleado_id fecha_ingreso fecha_ingreso_std
0        E001    15/01/2020        2020-01-15
1        E002    2021-03-10        2021-03-10
2        E003    10-02-2022        2022-02-10
3        E004    2019/12/05        2019-12-05
4        E005    2023-01-15        2023-01-15
5        E006    2020/08/20        2020-08-20


Normalización de Teléfonos

In [28]:
# Función para estandarizar números de teléfono
def estandarizar_telefono(telefono):
    """Estandariza formato de teléfono a XX-XXX-XXXX."""
    # Remover todo excepto dígitos
    digitos = re.sub(r'\D', '', str(telefono))
    
    # Formatear si tiene 9 dígitos (agregando código de área)
    if len(digitos) == 9:
        return f"91-{digitos[2:5]}-{digitos[5:]}"
    elif len(digitos) == 11 and digitos.startswith('91'):
        return f"91-{digitos[2:5]}-{digitos[5:]}"
    else:
        return telefono  # Devolver original si no se puede formatear

empleados_limpio['telefono_std'] = empleados_limpio['telefono'].apply(estandarizar_telefono)

print("Teléfonos estandarizados:")
print(empleados_limpio[['empleado_id', 'telefono', 'telefono_std']])

Teléfonos estandarizados:
  empleado_id     telefono telefono_std
0        E001  91-555-0123  91-555-0123
1        E002  91 555 0124  91-555-0124
2        E003    915550125  91-555-0125
3        E004  91.555.0126  91-555-0126
4        E005  91-555-0127  91-555-0127
5        E006    915550128  91-555-0128


Modificaciones Condicionales

Usando loc para Modificaciones Específicas

In [29]:
# Estandarizar estados usando loc
empleados_limpio['estado_std'] = empleados_limpio['estado'].str.lower()

# Mapear a valores estándar
empleados_limpio.loc[empleados_limpio['estado_std'] == 'activo', 'estado_std'] = 'Activo'
empleados_limpio.loc[empleados_limpio['estado_std'] == 'inactivo', 'estado_std'] = 'Inactivo'

print("Estados estandarizados:")
print(empleados_limpio[['empleado_id', 'estado', 'estado_std']])

Estados estandarizados:
  empleado_id    estado estado_std
0        E001    activo     Activo
1        E002    ACTIVO     Activo
2        E003    Activo     Activo
3        E004  inactivo   Inactivo
4        E005  INACTIVO   Inactivo
5        E006    Activo     Activo


Modificaciones Basadas en Multiple Condiciones

In [30]:
# Crear columna de categoría salarial basada en departamento
def categorizar_salario_por_dept(row):
    if row['departamento_normalizado'] == 'Consultoría':
        if row['salario_numerico'] >= 65000:
            return 'Senior'
        else:
            return 'Junior'
    elif row['departamento_normalizado'] == 'IT':
        if row['salario_numerico'] >= 68000:
            return 'Senior'
        else:
            return 'Junior'
    else:  # Finanzas
        if row['salario_numerico'] >= 70000:
            return 'Senior'
        else:
            return 'Junior'

empleados_limpio['categoria_salarial'] = empleados_limpio.apply(categorizar_salario_por_dept, axis=1)

print("Categorización salarial por departamento:")
print(empleados_limpio[['empleado_id', 'departamento_normalizado', 'salario_numerico', 'categoria_salarial']])


Categorización salarial por departamento:
  empleado_id departamento_normalizado  salario_numerico categoria_salarial
0        E001              Consultoría           65000.0             Senior
1        E002                       IT           70000.0             Senior
2        E003              Consultoría           62000.0             Junior
3        E004                 Finanzas           75000.0             Senior
4        E005                       IT           68000.0             Senior
5        E006              Consultoría           67000.0             Senior


Corrección de Datos Basada en Reglas

Validación y Corrección de Emails

In [31]:
# Función para validar formato de email
def es_email_valido(email):
    """Verifica si un email tiene formato válido."""
    patron = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return bool(re.match(patron, email))

# Identificar emails inválidos
empleados_limpio['email_valido'] = empleados_limpio['email'].apply(es_email_valido)

# Corregir emails problemáticos (ejemplo: falta de dominio completo)
def corregir_email(email):
    """Intenta corregir problemas comunes en emails."""
    if es_email_valido(email):
        return email
    
    # Corrección común: agregar .com si falta
    if '@company' in email and not '.com' in email:
        return email + '.com'
    
    return email

empleados_limpio['email_corregido'] = empleados_limpio['email'].apply(corregir_email)

print("Validación y corrección de emails:")
print(empleados_limpio[['empleado_id', 'email', 'email_valido', 'email_corregido']])


Validación y corrección de emails:
  empleado_id                        email  email_valido  \
0        E001       ana.garcia@company.com          True   
1        E002     carlos.lopez@company.com          True   
2        E003  maria.rodriguez@company.com          True   
3        E004       juan.perez@company.com          True   
4        E005     laura.martin@company.com          True   
5        E006    pedro.sanchez@company.com          True   

               email_corregido  
0       ana.garcia@company.com  
1     carlos.lopez@company.com  
2  maria.rodriguez@company.com  
3       juan.perez@company.com  
4     laura.martin@company.com  
5    pedro.sanchez@company.com  


Detección y Corrección de Outliers

In [32]:
# Detectar salarios que parecen outliers
q1 = empleados_limpio['salario_numerico'].quantile(0.25)
q3 = empleados_limpio['salario_numerico'].quantile(0.75)
iqr = q3 - q1
limite_inferior = q1 - 1.5 * iqr
limite_superior = q3 + 1.5 * iqr

# Marcar outliers
empleados_limpio['salario_outlier'] = (
    (empleados_limpio['salario_numerico'] < limite_inferior) |
    (empleados_limpio['salario_numerico'] > limite_superior)
)

print(f"Límites de salario: {limite_inferior:.0f} - {limite_superior:.0f}")
print("Detección de outliers salariales:")
print(empleados_limpio[['empleado_id', 'salario_numerico', 'salario_outlier']])

Límites de salario: 59500 - 75500
Detección de outliers salariales:
  empleado_id  salario_numerico  salario_outlier
0        E001           65000.0            False
1        E002           70000.0            False
2        E003           62000.0            False
3        E004           75000.0            False
4        E005           68000.0            False
5        E006           67000.0            False


Modificaciones en Lote

Usando replace() para Múltiples Reemplazos

In [33]:
# Crear dataset con abreviaciones inconsistentes
datos_proyectos = pd.DataFrame({
    'proyecto_id': ['P001', 'P002', 'P003', 'P004'],
    'estado': ['IP', 'En Progreso', 'COMPLETADO', 'comp'],
    'prioridad': ['H', 'Alta', 'MEDIA', 'm'],
    'tipo': ['Cons', 'Consultoría', 'TRAINING', 'train']
})

# Diccionarios de mapeo para estandarización
mapeo_estado = {
    'IP': 'En Progreso',
    'En Progreso': 'En Progreso',
    'COMPLETADO': 'Completado',
    'comp': 'Completado'
}

mapeo_prioridad = {
    'H': 'Alta',
    'Alta': 'Alta',
    'MEDIA': 'Media',
    'm': 'Media'
}

mapeo_tipo = {
    'Cons': 'Consultoría',
    'Consultoría': 'Consultoría',
    'TRAINING': 'Training',
    'train': 'Training'
}

# Aplicar mapeos
datos_proyectos['estado_std'] = datos_proyectos['estado'].replace(mapeo_estado)
datos_proyectos['prioridad_std'] = datos_proyectos['prioridad'].replace(mapeo_prioridad)
datos_proyectos['tipo_std'] = datos_proyectos['tipo'].replace(mapeo_tipo)

print("Estandarización de datos de proyectos:")
print(datos_proyectos)

Estandarización de datos de proyectos:
  proyecto_id       estado prioridad         tipo   estado_std prioridad_std  \
0        P001           IP         H         Cons  En Progreso          Alta   
1        P002  En Progreso      Alta  Consultoría  En Progreso          Alta   
2        P003   COMPLETADO     MEDIA     TRAINING   Completado         Media   
3        P004         comp         m        train   Completado         Media   

      tipo_std  
0  Consultoría  
1  Consultoría  
2     Training  
3     Training  


Usando map() para Transformaciones Complejas

In [34]:
# Crear códigos únicos para empleados basados en múltiples campos
def generar_codigo_empleado(row):
    """Genera código único: DEPT_YYYY_NNN"""
    dept_codigo = {
        'Consultoría': 'CONS',
        'IT': 'TECH',
        'Finanzas': 'FIN'
    }
    
    año_ingreso = row['fecha_ingreso_std'].year if pd.notna(row['fecha_ingreso_std']) else 2020
    dept = dept_codigo.get(row['departamento_normalizado'], 'UNK')
    numero = row.name + 1  # Usar índice + 1 como número
    
    return f"{dept}_{año_ingreso}_{numero:03d}"

empleados_limpio['codigo_empleado'] = empleados_limpio.apply(generar_codigo_empleado, axis=1)

print("Códigos de empleado generados:")
print(empleados_limpio[['empleado_id', 'departamento_normalizado', 'fecha_ingreso_std', 'codigo_empleado']])


Códigos de empleado generados:
  empleado_id departamento_normalizado fecha_ingreso_std codigo_empleado
0        E001              Consultoría        2020-01-15   CONS_2020_001
1        E002                       IT        2021-03-10   TECH_2021_002
2        E003              Consultoría        2022-02-10   CONS_2022_003
3        E004                 Finanzas        2019-12-05    FIN_2019_004
4        E005                       IT        2023-01-15   TECH_2023_005
5        E006              Consultoría        2020-08-20   CONS_2020_006


Casos Prácticos de Consultoría

Caso 1: Estandarización de Datos de Cliente

In [35]:
# Simular datos de clientes con inconsistencias
clientes_raw = pd.DataFrame({
    'cliente_id': ['C001', 'C002', 'C003', 'C004'],
    'nombre_empresa': ['Empresa A S.A.', 'EMPRESA B SA', 'empresa c s.a.', 'Empresa D S.A'],
    'industria': ['Tecnología', 'TECH', 'Finanzas', 'fin'],
    'tamaño': ['Grande', 'G', 'Mediana', 'P'],
    'ingresos': ['1.5M', '2,000,000', '500K', '100,000']
})

def estandarizar_datos_cliente(df):
    """Estandariza datos de cliente para consistencia."""
    df = df.copy()
    
    # Estandarizar nombres de empresa
    df['nombre_empresa'] = (df['nombre_empresa']
                           .str.title()
                           .str.replace(' S.A.', ' S.A.')
                           .str.replace(' Sa', ' S.A.')
                           .str.replace(' SA', ' S.A.'))
    
    # Mapear industrias
    mapeo_industria = {
        'Tecnología': 'Tecnología',
        'TECH': 'Tecnología',
        'Finanzas': 'Finanzas',
        'fin': 'Finanzas'
    }
    df['industria'] = df['industria'].replace(mapeo_industria)
    
    # Mapear tamaños
    mapeo_tamaño = {
        'Grande': 'Grande',
        'G': 'Grande',
        'Mediana': 'Mediana',
        'M': 'Mediana',
        'Pequeña': 'Pequeña',
        'P': 'Pequeña'
    }
    df['tamaño'] = df['tamaño'].replace(mapeo_tamaño)
    
    # Convertir ingresos a formato numérico
    def convertir_ingresos(ingreso_str):
        ingreso_str = str(ingreso_str).upper().replace(',', '')
        if 'M' in ingreso_str:
            return float(ingreso_str.replace('M', '')) * 1000000
        elif 'K' in ingreso_str:
            return float(ingreso_str.replace('K', '')) * 1000
        else:
            return float(ingreso_str)
    
    df['ingresos_numericos'] = df['ingresos'].apply(convertir_ingresos)
    
    return df

clientes_limpio = estandarizar_datos_cliente(clientes_raw)

print("Datos de clientes estandarizados:")
print(clientes_limpio)

Datos de clientes estandarizados:
  cliente_id  nombre_empresa   industria   tamaño   ingresos  \
0       C001  Empresa A S.A.  Tecnología   Grande       1.5M   
1       C002  Empresa B S.A.  Tecnología   Grande  2,000,000   
2       C003  Empresa C S.A.    Finanzas  Mediana       500K   
3       C004   Empresa D S.A    Finanzas  Pequeña    100,000   

   ingresos_numericos  
0           1500000.0  
1           2000000.0  
2            500000.0  
3            100000.0  


Caso 2: Actualización Masiva Basada en Nuevas Reglas

In [36]:
# Simular cambio en política salarial
def aplicar_nueva_politica_salarial(df):
    """Aplica nueva política salarial basada en desempeño y mercado."""
    df = df.copy()
    
    # Aumentos base por departamento (simulado)
    aumentos_base = {
        'Consultoría': 1.05,  # 5%
        'IT': 1.08,          # 8%
        'Finanzas': 1.03     # 3%
    }
    
    # Aplicar aumento base
    df['salario_nuevo'] = df.apply(
        lambda row: row['salario_numerico'] * aumentos_base.get(row['departamento_normalizado'], 1.0),
        axis=1
    )
    
    # Bonus adicional para empleados senior
    df.loc[df['categoria_salarial'] == 'Senior', 'salario_nuevo'] *= 1.02
    
    # Redondear a múltiplos de 1000
    df['salario_nuevo'] = (df['salario_nuevo'] / 1000).round() * 1000
    
    # Calcular aumento porcentual
    df['aumento_pct'] = ((df['salario_nuevo'] - df['salario_numerico']) / df['salario_numerico']) * 100
    
    return df

empleados_actualizados = aplicar_nueva_politica_salarial(empleados_limpio)

print("Aplicación de nueva política salarial:")
print(empleados_actualizados[['empleado_id', 'departamento_normalizado', 'categoria_salarial', 
                               'salario_numerico', 'salario_nuevo', 'aumento_pct']])

Aplicación de nueva política salarial:
  empleado_id departamento_normalizado categoria_salarial  salario_numerico  \
0        E001              Consultoría             Senior           65000.0   
1        E002                       IT             Senior           70000.0   
2        E003              Consultoría             Junior           62000.0   
3        E004                 Finanzas             Senior           75000.0   
4        E005                       IT             Senior           68000.0   
5        E006              Consultoría             Senior           67000.0   

   salario_nuevo  aumento_pct  
0        70000.0     7.692308  
1        77000.0    10.000000  
2        65000.0     4.838710  
3        79000.0     5.333333  
4        75000.0    10.294118  
5        72000.0     7.462687  
